In [1]:
import sys
import os

root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(),'..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [2]:
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data, procedure_data, update_data
from conexion_sap import connect_to_sap
import datetime
import pandas as pd
import numpy as np
import json

In [3]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [4]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
KEY = r'C:\Users\CGM\Projects\Actualizar_OC\config\key_cloud.json'
# Escribe aquí el ID de tu documento:
SPREADSHEET_ID = '1McMSLQAGpRq12vME_3CBuyKny64B31-0IeJuy6Jm_fQ'

creds = None
creds = service_account.Credentials.from_service_account_file(KEY, scopes=SCOPES)

service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

# Llamada a la api
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range='Precios!A:H').execute()
# Extraemos values del resultado
values = result.get('values',[])

In [5]:
cotizador_ipesa = pd.DataFrame(values)  

In [6]:
cotizador_ipesa.columns = cotizador_ipesa.iloc[1]
cotizador_ipesa = cotizador_ipesa[2:].reset_index(drop=True)

In [33]:
consulta = 'select identificador,material,reserva_centro,orden,ctd_dif from tbl_abastecimiento where estado_compra = "Por_pedir" and ate_accion="MERCADERIA" and ctd_dif > 0'

In [ ]:
con = crear_conexion()
materialesxcomprar = consult_data(con, consulta)
cerrar_conexion(con)

Conexion Exitosa


In [36]:
df_materialesxcomprar = pd.DataFrame(materialesxcomprar)

In [39]:
consulta_hist = 'select * from tbl_comprasR3S4_hist where version_sap="S4" '

In [40]:
con = crear_conexion()
historico_materiales = consult_data(con, consulta_hist)
cerrar_conexion(con)

Conexion Exitosa
conexion cerrada


In [42]:
df_historico_materiales = pd.DataFrame(historico_materiales)

In [3]:
data_prueba = pd.read_excel('data_prueba.xlsx')

In [4]:
grupos = data_prueba.groupby(['centro','cod_proveedor'])
for (centro,proveedor), pedidos_grupo in grupos:
    print(centro,proveedor)
    display(pedidos_grupo)
    print('-------------------')
    for index, row in pedidos_grupo.iterrows():
        print(row['material'],row['cantidad'])
        print(index)

C200 1200000011


,material,precio,cantidad,UND,cod_proveedor,Proveedor,Comprador,centro
0,JDTY26813_,139.05,20,GLN,1200000011,IPESA,Cesar Toro,C200
1,JDER263096,78.03,4,UN,1200000011,IPESA,Cesar Toro,C200
2,JDER263097,55.39,4,UN,1200000011,IPESA,Cesar Toro,C200
3,JD45M7063,11.55,4,UN,1200000011,IPESA,Cesar Toro,C200
4,JDAL205022,40.61,4,UN,1200000011,IPESA,Cesar Toro,C200
...,...,...,...,...,...,...,...,...
81,JDRE61631,92.93,1,UN,1200000011,IPESA,Cesar Toro,C200
82,JDRE61645,66.05,1,UN,1200000011,IPESA,Cesar Toro,C200
83,JDRE54847,95.93,1,UN,1200000011,IPESA,Cesar Toro,C200
84,JDER372511,591.46,1,UN,1200000011,IPESA,Cesar Toro,C200


-------------------
JDTY26813_ 20
0
JDER263096 4
1
JDER263097 4
2
JD45M7063 4
3
JDAL205022 4
4
JDAL65432 5
5
JDDZ111672 4
6
JDL156857 6
7
JDLVA12335 4
8
JDR119585 4
9
JDR125573 4
10
JDR141075 2
11
JDR141076 2
12
JDR141077 4
13
JDR141619 2
14
JDR172747 2
15
JDR233207 2
16
JDR270966 4
17
JDR270966  4
18
JDR515195 2
19
JDR522690 2
20
JDR524480 2
21
JDRE173315 2
22
jdRE190713 2
23
JDRE219008 2
24
JDRE227648 2
25
JDRE245479 2
26
JDRE255758 6
27
JDRE261354 1
28
JDRE271398 3
29
JDRE272374 3
30
JDRE45864 1
31
JDRE45944 2
32
JDRE502650 1
33
JDRE518097 2
34
JDRE529377  1
35
JDER394677 1
36
JDRE543483 1
37
JDRE60029 1
38
JDRE61827 1
39
JDSJ10714 2
40
JDSJ15809 1
41
JDSJ16496 1
42
JDSJ31628 1
43
JDSJ32011 1
44
JDRE173314 3
45
JDJD7147 3
46
JDJD9449 3
47
JDR114282 3
48
JDJD9345 2
49
JDLVU28068 2
50
JDSU44573 2
51
JDSJ14196 1
52
JDR126697 2
53
JDRE248610 2
54
JDSU51797 1
55
JDSU51795 1
56
JDRE274180 2
57
JDR204520 4
58
JDR138202 4
59
JDR257248 2
60
JDSU20740 2
61
JD40M7136 1
62
JDSU303919 1
63
JDSU2

In [ ]:
def get_db_config(path="config/credenciales_sap.json"):
    """Carga la configuración de la base de datos desde un archivo JSON."""
    with open(path, "r") as config_file:
        return json.load(config_file)
        
config = get_db_config()    
connection_name = config["sap_connection_name"]
user_sap = config["sap_user"]
pwa_sap = config["sap_password"]

session = connect_to_sap(connection_name=connection_name,user_sap=user_sap,pwa_sap=pwa_sap)
if session:
    grupos = data_prueba.groupby(['centro','cod_proveedor'])
    for (centro,proveedor), pedidos_grupo in grupos:
        session.findById("wnd[0]").maximize()
        session.findById("wnd[0]/tbar[0]/okcd").text = "ME21N"
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0013/subSUB0:SAPLMEGUI:0030/subSUB1:SAPLMEGUI:1105/cmbMEPO_TOPLINE-BSART").key = "ZNAC"
        session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0013/subSUB0:SAPLMEGUI:0030/subSUB1:SAPLMEGUI:1105/ctxtMEPO_TOPLINE-SUPERFIELD").text = proveedor
